# Scraping EdX

In [5]:
from bs4 import BeautifulSoup
import urllib.request
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

url = 'https://www.edx.org/course/?subject=Computer%20Science&language=English'

browser = webdriver.Chrome('E:/ITS - Teknik Informatika/SEMESTER 7/Tugas-Akhir/Master/App/chromedriver.exe')
browser.get(url)

SCROLL_PAUSE_TIME = 4.5

# Get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

soup = BeautifulSoup(browser.page_source,"html.parser")

all_=[]
j = soup.find_all('a', class_='course-link')
for link in j:
    all_.append(link.get('href'))
all_

['https://www.edx.org/course/computing-in-python-i-fundamentals-and-procedural-programming-0',
 'https://www.edx.org/course/computing-in-python-iv-objects-algorithms-0',
 'https://www.edx.org/course/computing-in-python-ii-control-structures-0',
 'https://www.edx.org/course/computing-in-python-iii-data-structures-0',
 'https://www.edx.org/course/introduction-to-java-programming-writing-good-code',
 'https://www.edx.org/course/introduction-java-programming-part-1-hkustx-comp102-1x-9',
 'https://www.edx.org/course/introduction-java-programming-part-2-hkustx-comp102-2x-9',
 'https://www.edx.org/course/introduction-mobile-application-hkustx-comp107x-8',
 'https://www.edx.org/course/introduction-to-management-information-systems-mis-a-survival-guide-0',
 'https://www.edx.org/course/the-software-architect-code-building-the-digital-world-0',
 'https://www.edx.org/course/javascript-introduction',
 'https://www.edx.org/course/dna-sequences-alignments-analysis-umuc-usmx-bif001x',
 'https://www.ed

In [7]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import json
from tqdm import tqdm

matkul = []
for link in tqdm(all_):

    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = Request(link,headers=hdr)
    page = urlopen(req)
    soup = BeautifulSoup(page)
    html = str(soup)
    d_start = html.find('"description":"')
    d_end = html.find('",', d_start)
    cp_start = html.find('"educationalOutcome":"')
    cp_end = html.find('",', cp_start)
    x={}
    x['nama_matkul'] = str(soup.find('title'))
    x['deskripsi_matkul'] = str(html[d_start+14:d_end+2])
    x['capaian_pembelajaran'] = str(html[cp_start+21:cp_end+2])
    
    matkul.append(x)

with open('Data/scrap/scrap_edx.json', 'w') as outfile:
    json.dump(matkul, outfile)


 27%|█████████████████████▉                                                          | 135/491 [04:42<10:08,  1.71s/it]


 55%|████████████████████████████████████████████▏                                   | 271/491 [08:51<10:13,  2.79s/it]


 83%|██████████████████████████████████████████████████████████████████▎             | 407/491 [14:02<02:20,  1.67s/it]


100%|████████████████████████████████████████████████████████████████████████████████| 491/491 [16:35<00:00,  1.79s/it]


In [2]:
import json
with open ('Data/scrap/scrap_edx.json') as f:
    data = json.load(f)
    
print(data[0])

{'nama_matkul': '<title data-react-helmet="true"></title>', 'deskripsi_matkul': '"Learn the fundamentals of computing in Python, including variables, operators, and writing and debugging your own programs.",', 'capaian_pembelajaran': ''}


# TRANSLATE

#### Mengambil dataset mooc

In [1]:
import json
nama=[]
deskripsi=[]
cp=[]
pb=[]
with open('Data/dataset_mooc.json',encoding="utf-8") as f:
    data_mooc = json.load(f)
print(data_mooc[207])

{'nama_matkul': 'CS50s Computer Science for Business Professionals', 'deskripsi_matkul': '"This is CS50\\u0026rsquo;s introduction to computer science for business professionals, designed for managers, product managers, founders, and decision-makers more generally. Whereas CS50 itself takes a bottom-up approach, emphasizing mastery of low-level concepts and implementation details thereof, this course takes a top-down approach, emphasizing mastery of high-level concepts and design decisions related thereto. Through lectures on computational thinking, programming languages, internet technologies, web development, technology stacks, and cloud computing, this course empowers you to make technological decisions even if not a technologist yourself. You\\u0026rsquo;ll emerge from this course with first-hand appreciation of how it all works and all the more confident in the factors that should guide your decision-making.",', 'capaian_pembelajaran': '" computational thinking programming languag

#### Melakukan case folding

In [3]:
import re
from tqdm import tqdm

cf = []
for contenttrans in tqdm(data_mooc):
    contentLower= {}
    contentClear = {}
    contentLower['nama_matkul'] = contenttrans['nama_matkul'].lower()
    contentLower['deskripsi_matkul'] = contenttrans['deskripsi_matkul'].lower()
    contentLower['capaian_pembelajaran'] = contenttrans['capaian_pembelajaran'].lower()
    contentLower['pokok_bahasan'] = contenttrans['pokok_bahasan'].lower() 
    contentClear['nama_matkul'] = re.sub(r'[^ a-zA-Z0-9.+]', ' ', str(contentLower['nama_matkul']))
    contentClear['deskripsi_matkul'] = re.sub(r'[^ a-zA-Z0-9.+]', ' ', str(contentLower['deskripsi_matkul']))
    contentClear['capaian_pembelajaran'] = re.sub(r'[^ a-zA-Z0-9.+]', ' ', str(contentLower['capaian_pembelajaran']))
    contentClear['pokok_bahasan'] = re.sub(r'[^ a-zA-Z0-9.+]', ' ', str(contentLower['pokok_bahasan']))
    cf.append(contentClear)
print (cf[130])

100%|████████████████████████████████████████| 376/376 [00:00<00:00, 37700.71it/s]


{'nama_matkul': 'devops testing', 'deskripsi_matkul': ' this course is part of the microsoft professional program in devops.in this course  you will learn about different test types  usage scenarios and the benefits of a test driven development approach. we will cover best practices around the usage of unit  api  integration   performance  functional  acceptance  automated  coded  ui and exploratory testing. you will learn how to create and analyze these tests using various tools such as visual studio team services  vsts   visual studio  application insights and third party frameworks such as selenium  ', 'capaian_pembelajaran': '  understand various testing types and usage scenarios test driven development and it u0026rsquo s benefits how to create and implement unit tests and the elements of a good test u0026nbsp  how to create basic api tests  or integration tests  performance testing and how to create a performance test using visual studio and visual studio team services  vsts  how

In [7]:
from textblob import TextBlob
import json
from tqdm import tqdm

matkul = []
for index in range (26, len(cf)):
    content=cf[index]
    mooc = {}
    print(content['nama_matkul'])
    dl_nm = TextBlob(content['nama_matkul']).detect_language()
    mooc['nama_matkul'] = TextBlob(content['nama_matkul']).translate(from_lang = dl_nm, to='id')
    mooc['deskripsi_matkul'] = TextBlob(content['deskripsi_matkul']).translate(from_lang =dl_nm, to='id')
    mooc['capaian_pembelajaran'] = TextBlob(content['capaian_pembelajaran']).translate(from_lang =dl_nm, to='id')
    matkul.append(mooc)
    print(matkul)

# with open('Data/dataset_mooc_tr.json', 'w') as outfile:
#     json.dump(matkul, outfile)
print(matkul[0])

computing in python iv  objects and algorithms
[{'nama_matkul': TextBlob("komputasi dalam objek python iv dan algoritma"), 'deskripsi_matkul': TextBlob("lengkapi pengetahuan pengantar Anda tentang ilmu komputer dengan kursus terakhir ini pada objek dan algoritme. Sekarang Anda telah belajar tentang struktur kontrol yang kompleks dan struktur data belajar untuk mengembangkan program yang lebih intuitif memanfaatkan pemahaman alami Anda tentang masalah melalui pemrograman berorientasi objek. kemudian belajar menganalisa kompleksitas dan efisiensi program-program ini melalui algoritma. Selain itu, beri tahukan pengetahuan Anda yang lebih luas tentang pengantar komputasi dengan ujian komprehensif. Pada akhir kursus ini, Anda akan dapat menulis program dalam python yang memanfaatkan pemahaman Anda yang lebih alami tentang struktur data dengan membuat objek untuk merepresentasikan struktur tempat Anda bekerja dengan sebagian besar sering. misalnya jika Anda membuat aplikasi daftar kelas, And

[{'nama_matkul': TextBlob("komputasi dalam objek python iv dan algoritma"), 'deskripsi_matkul': TextBlob("lengkapi pengetahuan pengantar Anda tentang ilmu komputer dengan kursus terakhir ini pada objek dan algoritme. Sekarang Anda telah belajar tentang struktur kontrol yang kompleks dan struktur data belajar untuk mengembangkan program yang lebih intuitif memanfaatkan pemahaman alami Anda tentang masalah melalui pemrograman berorientasi objek. kemudian belajar menganalisa kompleksitas dan efisiensi program-program ini melalui algoritma. Selain itu, beri tahukan pengetahuan Anda yang lebih luas tentang pengantar komputasi dengan ujian komprehensif. Pada akhir kursus ini, Anda akan dapat menulis program dalam python yang memanfaatkan pemahaman Anda yang lebih alami tentang struktur data dengan membuat objek untuk merepresentasikan struktur tempat Anda bekerja dengan sebagian besar sering. misalnya jika Anda membuat aplikasi daftar kelas, Anda akan belajar cara membuat objek yang mewakili

the software architect code  building the digital world
[{'nama_matkul': TextBlob("komputasi dalam objek python iv dan algoritma"), 'deskripsi_matkul': TextBlob("lengkapi pengetahuan pengantar Anda tentang ilmu komputer dengan kursus terakhir ini pada objek dan algoritme. Sekarang Anda telah belajar tentang struktur kontrol yang kompleks dan struktur data belajar untuk mengembangkan program yang lebih intuitif memanfaatkan pemahaman alami Anda tentang masalah melalui pemrograman berorientasi objek. kemudian belajar menganalisa kompleksitas dan efisiensi program-program ini melalui algoritma. Selain itu, beri tahukan pengetahuan Anda yang lebih luas tentang pengantar komputasi dengan ujian komprehensif. Pada akhir kursus ini, Anda akan dapat menulis program dalam python yang memanfaatkan pemahaman Anda yang lebih alami tentang struktur data dengan membuat objek untuk merepresentasikan struktur tempat Anda bekerja dengan sebagian besar sering. misalnya jika Anda membuat aplikasi daftar k

javascript introduction
[{'nama_matkul': TextBlob("komputasi dalam objek python iv dan algoritma"), 'deskripsi_matkul': TextBlob("lengkapi pengetahuan pengantar Anda tentang ilmu komputer dengan kursus terakhir ini pada objek dan algoritme. Sekarang Anda telah belajar tentang struktur kontrol yang kompleks dan struktur data belajar untuk mengembangkan program yang lebih intuitif memanfaatkan pemahaman alami Anda tentang masalah melalui pemrograman berorientasi objek. kemudian belajar menganalisa kompleksitas dan efisiensi program-program ini melalui algoritma. Selain itu, beri tahukan pengetahuan Anda yang lebih luas tentang pengantar komputasi dengan ujian komprehensif. Pada akhir kursus ini, Anda akan dapat menulis program dalam python yang memanfaatkan pemahaman Anda yang lebih alami tentang struktur data dengan membuat objek untuk merepresentasikan struktur tempat Anda bekerja dengan sebagian besar sering. misalnya jika Anda membuat aplikasi daftar kelas, Anda akan belajar cara mem

deep learning with tensorflow
[{'nama_matkul': TextBlob("komputasi dalam objek python iv dan algoritma"), 'deskripsi_matkul': TextBlob("lengkapi pengetahuan pengantar Anda tentang ilmu komputer dengan kursus terakhir ini pada objek dan algoritme. Sekarang Anda telah belajar tentang struktur kontrol yang kompleks dan struktur data belajar untuk mengembangkan program yang lebih intuitif memanfaatkan pemahaman alami Anda tentang masalah melalui pemrograman berorientasi objek. kemudian belajar menganalisa kompleksitas dan efisiensi program-program ini melalui algoritma. Selain itu, beri tahukan pengetahuan Anda yang lebih luas tentang pengantar komputasi dengan ujian komprehensif. Pada akhir kursus ini, Anda akan dapat menulis program dalam python yang memanfaatkan pemahaman Anda yang lebih alami tentang struktur data dengan membuat objek untuk merepresentasikan struktur tempat Anda bekerja dengan sebagian besar sering. misalnya jika Anda membuat aplikasi daftar kelas, Anda akan belajar ca

statistical analysis in bioinformatics
[{'nama_matkul': TextBlob("komputasi dalam objek python iv dan algoritma"), 'deskripsi_matkul': TextBlob("lengkapi pengetahuan pengantar Anda tentang ilmu komputer dengan kursus terakhir ini pada objek dan algoritme. Sekarang Anda telah belajar tentang struktur kontrol yang kompleks dan struktur data belajar untuk mengembangkan program yang lebih intuitif memanfaatkan pemahaman alami Anda tentang masalah melalui pemrograman berorientasi objek. kemudian belajar menganalisa kompleksitas dan efisiensi program-program ini melalui algoritma. Selain itu, beri tahukan pengetahuan Anda yang lebih luas tentang pengantar komputasi dengan ujian komprehensif. Pada akhir kursus ini, Anda akan dapat menulis program dalam python yang memanfaatkan pemahaman Anda yang lebih alami tentang struktur data dengan membuat objek untuk merepresentasikan struktur tempat Anda bekerja dengan sebagian besar sering. misalnya jika Anda membuat aplikasi daftar kelas, Anda akan b

html5 apps and games
[{'nama_matkul': TextBlob("komputasi dalam objek python iv dan algoritma"), 'deskripsi_matkul': TextBlob("lengkapi pengetahuan pengantar Anda tentang ilmu komputer dengan kursus terakhir ini pada objek dan algoritme. Sekarang Anda telah belajar tentang struktur kontrol yang kompleks dan struktur data belajar untuk mengembangkan program yang lebih intuitif memanfaatkan pemahaman alami Anda tentang masalah melalui pemrograman berorientasi objek. kemudian belajar menganalisa kompleksitas dan efisiensi program-program ini melalui algoritma. Selain itu, beri tahukan pengetahuan Anda yang lebih luas tentang pengantar komputasi dengan ujian komprehensif. Pada akhir kursus ini, Anda akan dapat menulis program dalam python yang memanfaatkan pemahaman Anda yang lebih alami tentang struktur data dengan membuat objek untuk merepresentasikan struktur tempat Anda bekerja dengan sebagian besar sering. misalnya jika Anda membuat aplikasi daftar kelas, Anda akan belajar cara membua

cyber security basics  a hands on approach


KeyboardInterrupt: 